In [1]:
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\Nick\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


# NASA Mars News Scrape

In [3]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

In [4]:
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [5]:
# retreive news title & news paragraph text
news_title = soup.find('div', class_='bottom_gradient').text
news_p = soup.find('div', class_='article_teaser_body').text
print(news_title)
print('---------------------------------')
print(news_p)

Another First: Perseverance Captures the Sounds of Driving on Mars
---------------------------------
NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.


# JPL Mars Space Images Scrape

In [6]:
# set up splinter with url for scraping
space_img_url = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html'
img_url_prefix = 'https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/'
browser.visit(space_img_url)
space_img_html = browser.html
soup2 = BeautifulSoup(space_img_html, 'html.parser')

In [7]:
# find featured image
img_result = soup2.find('img',class_='headerimage')
# attach full url to image & print link
feat_image_url = img_url_prefix + img_result['src']
print(feat_image_url)

https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/image/featured/mars1.jpg


# Mars Facts Table Scrape to HTML string

In [8]:
# Read table with Pandas
table_url = 'https://space-facts.com/mars/'
mars_table_html = pd.read_html(table_url, attrs={'id':'tablepress-p-mars-no-2'})

# Mars table to HTML string
mars_table_df = mars_table_html[0]
print(mars_table_df.to_html())

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
      <th>1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
      <td>Recorded

# Mars Hemispheres

In [101]:
hemisphere_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemisphere_url)
hem_html = browser.html
hem_soup = BeautifulSoup(hem_html,'html.parser')

In [103]:
hem_img_url = []
hem_divs = hem_soup.find_all('div',class_='description')
for div in hem_divs:
    img_title = div.find('a').text
    try:
        browser.click_link_by_partial_text(img_title)
        browser.click_link_by_partial_text('Sample')
    except:
        print('not found')
    # Grab new window url
    new_url = browser.windows[1].url
    # Close new window
    browser.windows[1].close()
    # Visit new url
    browser.visit(new_url)
    # Grab html and image src url
    html = browser.html
    soup = BeautifulSoup(html,'html.parser')
    img_tag = soup.find('img')
    full_img = img_tag['src']
    # save url to list
    hem_dict = {
        'Title': img_title,
        'img_url': full_img
    }
    hem_img_url.append(hem_dict)
    # Go back to page with mars image list
    browser.back()
    browser.back()

In [104]:
for i in hem_img_url:
    print(i)
    print('----------------')

{'Title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}
----------------
{'Title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}
----------------
{'Title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}
----------------
{'Title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}
----------------
